**LIKENESS SCORE**

This is an example of computing the Likeness Score: LS on datasets of real images and generated images by GANs.
It includes codes to compute the *2-class* distance-based separability index (DSI). There are two versions (CPU and GPU) of DSI.

Inputs:             Two folders have real images and generated images

Related paper:      A Novel Measure to Evaluate Generative Adversarial Networks Based on Direct Analysis of Generated Images
                    [In press] Neural Computing and Applications, 2021
                    https://arxiv.org/abs/2002.12345

In [ ]:
# import packages

import glob
import numpy as np
import scipy.misc
from scipy.spatial.distance import minkowski
from scipy.stats import ks_2samp
import time,imageio, os
import torch
from PIL import Image
from tqdm import tqdm
import cv2

from google.colab import drive, auth
import gspread
from google.auth import default

In [ ]:
# Authentification
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# link notebook to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# process generated input images
def get_gen_image_vector(filename):
    # im = imageio.imread(filename, pilmode = 'RGB')
    # using Image class instead
    im = Image.open(filename)
    # resize all images to (128, 128)
    resized_image = im.resize((128, 128))
    # convert image to numpy array
    image_array = np.array(resized_image)
    # DEBUGGING: shape of image array
    print(image_array.shape)
    # ISSUE: generated images have 1 channel, but real images have 3 channels bc they are RGB
    # Solution: convert all images to 1 channel grayscale


    return np.float32(np.ndarray.flatten(image_array))/255

In [ ]:
# process real input images
def get_real_image_vector(filename):
    # im = imageio.imread(filename, pilmode = 'RGB')
    # using Image class instead
    im = Image.open(filename)
    # resize all images to (128, 128)
    resized_image = im.resize((128, 128))
    # convert image to numpy array
    image_array = np.array(resized_image)
    # convert RGB image to grayscale image, needs an array input
    gray_image = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
    # DEBUGGING: shape of image array
    print(gray_image.shape)

    return np.float32(np.ndarray.flatten(gray_image))/255

In [ ]:
# CPU Version
def dists(data):  # compute ICD
    num = data.shape[0]
    data = data.reshape((num, -1))
    dist = []
    for i in tqdm(range(0,num-1)):
        for j in range(i+1,num):
            dist.append(minkowski(data[i],data[j]))

    return np.array(dist)

def dist_btw(a,b):  # compute BCD
    a = a.reshape((a.shape[0], -1))
    b = b.reshape((b.shape[0], -1))
    dist = []
    for i in tqdm(range(a.shape[0])):
        for j in range(b.shape[0]):
            dist.append(minkowski(a[i],b[j]))

    return np.array(dist)


def LS(real,gen):  # KS distance btw ICD and BCD
    dist_real = dists(real)  # ICD 1
    dist_gen = dists(gen)  # ICD 2
    distbtw = dist_btw(real, gen)  # BCD

    D_Sep_1, _ = ks_2samp(dist_real, distbtw)
    D_Sep_2, _ = ks_2samp(dist_gen, distbtw)

    return 1- np.max([D_Sep_1, D_Sep_2])  # LS=1-DSI

In [ ]:
# don't run for now
# GPU Version
def gpu_LS(real,gen):
    # to torch tensors
    t_gen = torch.from_numpy(gen)
    t_real = torch.from_numpy(real)

    dist_real = torch.cdist(t_real, t_real)  # ICD 1
    dist_real = torch.flatten(torch.tril(dist_real, diagonal=-1))  # remove repeats
    dist_real = dist_real[dist_real.nonzero()].flatten()  # remove distance=0 for distances btw same data points

    dist_gen = torch.cdist(t_gen, t_gen)  # ICD 2
    dist_gen = torch.flatten(torch.tril(dist_gen, diagonal=-1))  # remove repeats
    dist_gen = dist_gen[dist_gen.nonzero()].flatten()  # remove distance=0 for distances btw same data points

    distbtw = torch.cdist(t_gen, t_real)  # BCD
    distbtw = torch.flatten(distbtw)

    D_Sep_1, _ = ks_2samp(dist_real, distbtw)
    D_Sep_2, _ = ks_2samp(dist_gen, distbtw)

    return 1- np.max([D_Sep_1, D_Sep_2])  # LS=1-DSI

In [ ]:
# don't run for now
# main function
"""
if __name__ == '__main__':

    AbsLoc = '/content/gdrive/MyDrive/Leave/UW/CSE455/FinalProject/Evaluation'

    filenames_1 = glob.glob(os.path.join(AbsLoc, 'Generated/*.png'))
    gen = np.array([get_image_vector(filename) for filename in filenames_1])

    filenames_2 = glob.glob(os.path.join(AbsLoc, 'Real/*.png'))
    real = np.array([get_image_vector(filename) for filename in filenames_2])

    print('real #:   '+str(len(real)))
    print('gen #:   '+str(len(gen)))

    print('\n', 'LS= ', LS(real, gen))  # CPU ver.
    # print('\n', 'LS= ', gpu_LS(real, gen))  # GPU ver.
  """

In [ ]:
# force remount again
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# run this main function instead

# set path to Evaluation Generation Dataset
# %cd /content/gdrive/MyDrive/Leave/UW/CSE455/FinalProject/Evaluation/Generated_GAN/
%cd /content/gdrive/MyDrive/Leave/UW/CSE455/FinalProject/Evaluation/generated_images_vae_10000/
generated_files = glob.glob('*.png')
# filenames_1 = glob.glob(os.path.join(AbsLoc, 'Generated/*.png'))
# GAN files are RGB so they need to be converted to grayscale
# gen = np.array([get_real_image_vector(filename) for filename in generated_files])
gen = np.array([get_gen_image_vector(filename) for filename in generated_files])

# set path to Evaluation Real Dataset
%cd /content/gdrive/MyDrive/Leave/UW/CSE455/FinalProject/Evaluation/Real/
real_files = glob.glob('*.jpg')
# edit all of the real images so they have the same dimensions

# filenames_2 = glob.glob(os.path.join(AbsLoc, 'Real/*.png'))
real = np.array([get_real_image_vector(filename) for filename in real_files])

print('real #:   '+str(len(real)))
print('gen #:   '+str(len(gen)))

print('\n', 'LS= ', LS(real, gen))  # CPU ver.
# print('\n', 'LS= ', gpu_LS(real, gen))  # GPU ver.

/content/gdrive/MyDrive/Leave/UW/CSE455/FinalProject/Evaluation/generated_images_vae_10000
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
/content/gdrive/MyDrive/Leave/UW/CSE455/FinalProject/Evaluation/Real
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
real #:   10
gen #:   10


100%|██████████| 10/10 [00:00<00:00, 2333.28it/s]


 LS=  0.4588888888888889
